### 3-21
- Q1 copy 可以不copy内存 clone
- Q2 对哪维度求和就是消除哪一维度
- Q3 sum(axis=[0, 1])怎么求

## 自动求导
假设我们对函数 `y = 2x^T x `关于列向量x求导


In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
x.requires_grad_(True)
x.grad

In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [7]:
x.grad == 4 * x

tensor([True, True, True, True])

In [8]:
# 默认情况下, pytorch会累积梯度, 我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward() # y = x1 + x2 + x3 + x4
x.grad

tensor([1., 1., 1., 1.])

In [9]:
x.grad.zero_()
y = x * x # vector
y.sum().backward() # y = x1 + x2 + x 3 + x4
x.grad

tensor([0., 2., 4., 6.])

In [21]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

在 PyTorch 中，detach() 是一个用于从计算图中分离张量（tensor）的方法。它返回一个新的张量，该张量与原始张量具有相同的值，但不会记录梯度信息，意味着它不会参与梯度计算和反向传播。

In [22]:
y, u, z

(tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>),
 tensor([0., 1., 4., 9.]),
 tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>))

In [24]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

运行两次上述问题会出现  Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

尝试在图中第二次进行反向传播（或在已释放的张量之后直接访问已保存的张量）。当你调用 .backward() 或 autograd.grad() 时，图中的中间保存值会被释放。如果你需要在图中第二次进行反向传播，或者在调用 backward 后仍需访问保存的张量，请指定 retain_graph=True

即使构建函数的计算图需要通过 Python控制流(例如,崇件…"循环或任意函数调用)，我们仍然可以计算得到的变量的梯度

In [28]:
def f(a):
    b = a * 2
    
    while b.norm() < 1000: # |b| < 1000
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c=100 *b
    return c
    # size为空的话就是一个标量
a = torch.randn(size=(),requires_grad=True)
d = f(a)
d.backward()
a.grad == d / a

tensor(True)

In [31]:
d / a

tensor(1024., grad_fn=<DivBackward0>)